In [1]:
### Utenti Validi
# '2c938084683d9f87016d1636f55a00de': (u'Grazia Benedetti',True,"24.10.19","1.4.20")
# '2c938084683d9f87016ff0d54c3d0138': (u'Maria Luisa  Bortoluzzi',True,"29.1.20","1.5.20")
# '2c938084683d9f87016fd2ac0d240134': (u'Silvia Francescangeli',True,"23.1.20","1.5.20")
# '2c938084683d9f87016cd1cbb80100a7': (u'Enrico Vottero',True,"30.09.19","20.12.19")
# '2c938084683d9f87016d163666b900db': (u'Giorgio Bertolani',True,"9.10.19","20.12.19")}

### Setup delle librerie e connessione al database

In [2]:
import pymongo 
from pymongo import MongoClient
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import math
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import pickle

In [3]:
### connect to the db 
myclient = pymongo.MongoClient("mongodb://localhost:27018/")
mydb = myclient["mls"]
mycol = mydb["mqttlogs"]

### select a user and find all the respective entries
utente='2c938084683d9f87016cd1cbb80100a7'
print("PROCESSING USER:\n" + utente)

query = {'userid':utente}
db_entries = mycol.find(query)

PROCESSING USER:
2c938084683d9f87016cd1cbb80100a7


### Calcolo di interventions e acks

In [4]:
ordered_interventions_id = [] # an array with the intervention_id
interventions = dict() # a dictionary with all the interventions; the intervention_id is the key
acks = dict() # a dictionary with all the acks; the intervention_id is the key
escape_interventions = ["641056j1-0g78-vjqf-cpju-7dnx895011y9"]

print(datetime.now())

2023-12-30 15:03:59.035307


In [5]:
print("start querying DB for interventions and acks")

for d in db_entries :
    try :
        # collect all the user's interventions
        if utente in d['topic'] and "interventions" in d['topic'] and d['raw_payload']['id'] not in escape_interventions:
            ordered_interventions_id.append(d['raw_payload']['id']) 
            interventions[d['raw_payload']['id']] = d 

        # collect all the user's acks
        if utente in d['topic'] and "ack" in d['topic'] and d['raw_payload']['intervention_id'] in interventions.keys() :
            acks[d['raw_payload']['intervention_id']] = d
    # for bd_entries that are neither interventions nor acks
    except Exception as e:
        print(e,d)

print("end processing interventions and acks")
print(len(ordered_interventions_id),len(interventions),len(acks))  

start querying DB for interventions and acks
'intervention_id' {'_id': ObjectId('5d9b424eebaf7e0a71da563e'), 'topic': 'movecare/ack/2c938084683d9f87016cd1cbb80100a7/INFO', 'category': 'ack', 'userid': '2c938084683d9f87016cd1cbb80100a7', 'raw_payload': {'data': {'code': 'OS', 'location': '', 'objectid': '000000000000000000000A81', 'response': 'Found', 'roomid': 'empty'}, 'id': '2e5a5e13-68aa-4b33-a2dc-31f37044792e', 'time': {'t': '1570456140.641847496', 'temporality': 'timestamp'}, 'trace': {'OSresult': 'yes', 'objectid': '000000000000000000000A81', 'objectlocation': '[-1.72417130703,-0.921335104026,0.0222753854406]', 'objectspacelocation': ''}, 'userid': '2c938084683d9f87016cd1cbb80100a7'}, 'created_at': datetime.datetime(2019, 10, 7, 13, 49, 2, 204000), 'updatedAt': datetime.datetime(2019, 10, 7, 13, 49, 2, 204000), '__v': 0}
'intervention_id' {'_id': ObjectId('5d9b45e0ebaf7e0a71da59d3'), 'topic': 'movecare/ack/2c938084683d9f87016cd1cbb80100a7/INFO', 'category': 'ack', 'userid': '2c93

### Calcolo delle interventions dangling

In [6]:
timedeltas = dict()
responses = dict()
traces = dict()
times = dict() # contains timestamps about the beginning of an intervention and its ack, the key is the intervention_id
dangling_interventions = []

In [7]:
for i in interventions.keys() : 
	item = interventions[i]
	if i in acks and i in interventions :
		# timedeltas[i] = float(acks[i]['raw_payload']['time']['t']) - float(interventions[i]['raw_payload']['time']['t'])
		# responses[i] = acks[i]['raw_payload']['data']['response']
		# if "trace" in acks[i]['raw_payload'] :
			# traces[i] = acks[i]['raw_payload']['trace']
		times[i] = []
		times[i].append(float(interventions[i]['raw_payload']['time']['t']))
		times[i].append(float(acks[i]['raw_payload']['time']['t']))	
	else :
		dangling_interventions.append(i) 
print("end arranging intervention")
print(str(len(dangling_interventions))+ " dangling interventions")

end arranging intervention
34 dangling interventions


### Salvataggio dei dati raccolti sulle interventions

In [8]:
data_file = open("./data_interventions_"+str(utente)+".pkl", "wb")
pickle.dump([ordered_interventions_id,interventions,acks,dangling_interventions,times],data_file)
data_file.close()

### Calcolo delle run del robot

In [9]:
topic = 'movecare/environment/'+utente+'/ROBOT'
query = {'userid':utente, 'topic': topic, 'category':'environment'}
entries = mycol.find(query)

t=[]
poses=[]
poses_r = []
run = [] 

In [10]:
print("starting parsing entries")
for p in entries : # poses contain all the db entries having 'environment/ROBOT' as their topic
    if utente in p['topic'] and 'ROBOT' in p['topic']:
        t.append(p['raw_payload']['time']['t'])
        poses.append(p)
print("end of parsing entries")

print("defining runs")
for j in ordered_interventions_id:
    if j not in dangling_interventions : # selected all the non pending intervention_id
        poses_r = []
        for i in range(len(t)-1) :
            # if the time t of the pose at index i (from 0 to let(t)-1 is included between the start of an intervention and its ack, then update
            if float(t[i]) <= float(times[j][1]) and  float(t[i]) >= float(times[j][0]) :
                poses[i]["intervention_id"] = j
                poses_r.append(poses[i])
        # using only runs longer than 10 seconds and filter those longer than 1h cause are errors (We have runs lasting months)
        if len(poses_r) > 10 and len(poses_r) < 3600:
            run.append(poses_r)

starting parsing entries
end of parsing entries
defining runs


In [11]:
len(run)

608

### Salvataggio delle run

In [12]:
data_file = open("./run_fullcomunication_"+str(utente)+".pkl", "wb")
pickle.dump(run,data_file)
data_file.close()